## Campus Devcon - FEU Tech

### Sample Implementations of OpenAI Projects

In [1]:
import os
import requests

import openai
from transformers import CLIPProcessor, CLIPModel

from IPython.display import Image
from IPython.core.display import HTML 

import PIL

import numpy as np

openai.api_key = os.getenv("OPENAI_API_KEY")

### Load some functions

In [2]:
def generate_image(prompt = ""):

    response = openai.Image.create(
      prompt = prompt,
      n=1,
      size="256x256"
    )
    
    image_url = response['data'][0]['url']

    return image_url


def generate_image_variation(image_location):

    response = openai.Image.create_variation(
      image=open(image_location, "rb"),
      n=1,
      size="256x256"
    )
    image_url = response['data'][0]['url']   

    return image_url

def edit_image(image_location, prompt):

    response = openai.Image.create_edit(
        image = open(image_location, "rb"),
        mask = open("data/transparent_mask.png", "rb"),
        prompt= prompt,
        n=1,
        size="256x256"
    )
    image_url = response['data'][0]['url']
    return image_url

def load_raw_image(image_location):
    return PIL.Image.open(image_location)

def save_image(image_url, file_location):
    
    img_data = requests.get(image_url).content
    
    with open(file_location, 'wb') as handler:
        handler.write(img_data)

def get_image(image_url):
    return Image(url= image_url)

## Generate Images

In [3]:
dog_url = generate_image(prompt = "playful dog")
get_image(image_url = dog_url)

In [4]:
save_image(image_url = dog_url, file_location = "data/dog.png")

In [5]:
another_dog_url = generate_image_variation(image_location = "data/dog.png")
get_image(image_url = another_dog_url)

In [6]:
save_image(image_url = another_dog_url, file_location = "data/dog1.png")

In [7]:
dog_and_cat_url = generate_image(prompt = "dog and cat")
get_image(image_url = dog_and_cat_url)

In [8]:
save_image(image_url = dog_and_cat_url, file_location = "data/cat_and_dog.png")

In [10]:
edited_cat_and_dog_url = edit_image(
    image_location = "data/cat_and_dog.png", 
    prompt = "dog and cat are playing in garden")
get_image(image_url = edited_cat_and_dog_url)

In [11]:
save_image(image_url = dog_and_cat_url, file_location = "data/cat_and_dog_garden.png")

## Pretrained Models

## Description

The CLIP model was developed by researchers at OpenAI to learn about what contributes to robustness in computer vision tasks. The model was also developed to test the ability of models to generalize to arbitrary image classification tasks in a zero-shot manner. It was not developed for general model deployment - to deploy models like CLIP, researchers will first need to carefully study their capabilities in relation to the specific context they’re being deployed within.

The base model uses a ViT-L/14 Transformer architecture as an image encoder and uses a masked self-attention Transformer as a text encoder. These encoders are trained to maximize the similarity of (image, text) pairs via a contrastive loss.

The original implementation had two variants: one using a ResNet image encoder and the other using a Vision Transformer. This repository has the variant with the Vision Transformer.

See more: https://huggingface.co/openai/clip-vit-large-patch14

In [12]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [13]:
dog_image = load_raw_image("data/dog1.png")

In [14]:
def get_prediction(image, labels):

    inputs = processor(
        text           = labels, 
        images         = image, 
        return_tensors = "pt", 
        padding        = True
    )

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    probs = logits_per_image.softmax(dim = 1) 
    probs = probs.detach().cpu().numpy().tolist()[0]

    results = dict(zip(labels,probs))

    return results

In [15]:
get_prediction(image = dog_image, labels = ["a photo of a cat", "a photo of a dog"])

{'a photo of a cat': 0.0020531299524009228,
 'a photo of a dog': 0.9979469180107117}

In [16]:
get_prediction(image = dog_image, labels = ["dog", "cat"])

{'dog': 0.9997114539146423, 'cat': 0.0002885090361814946}

In [17]:
catdog_image = load_raw_image("data/cat_and_dog.png")
get_prediction(image = catdog_image, labels = ["dog", "cat"])

{'dog': 0.5237144231796265, 'cat': 0.4762856364250183}

## Natural language Processing

In [18]:
statement = """
  {0}
  
  I never imagined that I would find myself in a situation where I have to explain and defend my decision to leave a marriage, as well as justify the state of my mental health.

  It is truly disheartening that I am being questioned and asked to prove my worth as a songwriter and artist. However, despite the overwhelming circumstances, I feel a strong inner calling to stand up for myself and protect my integrity.

  I want to set things straight once and for all. Let me be clear: I have never employed a ghostwriter. Throughout my career, I have been fortunate enough to collaborate with immensely talented artists who treat each other with respect and acknowledge the contributions we all bring to the table. 

  Every song I have created is a true reflection of my deepest emotions and experiences. I am fully prepared to provide evidence that supports this truth, including screenshots of conversations and recordings of my songs, which will undoubtedly validate and disprove the harmful accusations made against me.

  I did not cheat on Jason. It is unfair that I find myself in a position where I have to defend my name and even explain why I did not deserve to be cheated on.

  Currently, my focus is on the path towards healing, as the weight of trauma still lingers heavily in my heart. This process is not only essential for my own well-being but also for the well-being of those dear to me who have unwaveringly stood by my side throughout these challenging times-my loving family, my dedicated band, my friends, who have all been my source of emotional support.

  Their presence has been my anchor, grounding me during these turbulent moments. I hold onto hope that, one day, when the dust settles and clarity is restored, I will have the capacity to embrace a love that is faithful and genuine. I take solace in the knowledge that I am Moira-an artist-who will bravely navigate these challenges and emerge stronger than ever.
"""

In [19]:
sentiment_analysis_prompt = statement.format("Classify the sentiment of the following statement:")

In [21]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt= sentiment_analysis_prompt,
  temperature=0,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

response["choices"][0]["text"]

'\nThe sentiment of this statement is hopeful and determined.'

In [22]:
sentiment_analysis_prompt = statement.format("Classify as positive or negative sentiment the following statement:")

In [23]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt= sentiment_analysis_prompt,
  temperature=0,
  max_tokens=60,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

response["choices"][0]["text"]

'\nThis statement is positive sentiment.'

In [24]:
summarize_prompt = statement.format("Summarize the following statement into strictly 2-3 sentences:")

In [25]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt= summarize_prompt,
  temperature=0,
  max_tokens=120,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

response["choices"][0]["text"]

'\nI am facing difficult circumstances as I have to explain and defend my decision to leave a marriage and justify my mental health. I am determined to stand up for myself and protect my integrity, and I am prepared to provide evidence that I have never employed a ghostwriter. I am focusing on healing and leaning on the support of my loving family, band, and friends. I am hopeful that I will be able to embrace a faithful and genuine love in the future.'

In [26]:
generate_image_code = """
    {0}
    
    def generate_image(prompt = ""):
    
        response = openai.Image.create(
          prompt = prompt,
          n=1,
          size="256x256"
        )
        
        image_url = response['data'][0]['url']
    
        return image_url
"""

In [27]:
code_prompt = generate_image_code.format("Annotate the following code:")

In [28]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt = code_prompt,
  temperature=0,
  max_tokens=120,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

print(response["choices"][0]["text"])


This function takes in an optional prompt string as an argument and uses the OpenAI Image API to generate an image with a size of 256x256. It then returns the URL of the generated image.


In [29]:
code_prompt = generate_image_code.format("Update the following code with docstring:")

response = openai.Completion.create(
  model="text-davinci-003",
  prompt = code_prompt,
  temperature=0,
  max_tokens=250,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

print(response["choices"][0]["text"])

    
def generate_image(prompt = ""):
    """
    Generates an image based on the given prompt.
    
    Parameters
    ----------
    prompt : str
        The prompt to generate the image from.
    
    Returns
    -------
    image_url : str
        The URL of the generated image.
    """
    response = openai.Image.create(
      prompt = prompt,
      n=1,
      size="256x256"
    )
    
    image_url = response['data'][0]['url']

    return image_url


### Final Code

In [30]:
def generate_image(prompt = ""):
    """
    Generates an image based on the given prompt.
    
    Parameters
    ----------
    prompt : str
        The prompt to generate the image from.
    
    Returns
    -------
    image_url : str
        The URL of the generated image.
    """
    response = openai.Image.create(
      prompt = prompt,
      n=1,
      size="256x256"
    )
    
    image_url = response['data'][0]['url']

    return image_url

In [35]:
prompt = statement.format("Extract keywords from following statement and create a prompt to create image based on the top 3 derived keywords:")

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= prompt,
  temperature=0,
  max_tokens=120,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

sentiment_prompt = response["choices"][0]["text"]

print(sentiment_prompt)

image_from_sentiment_url = generate_image(prompt = sentiment_prompt)
get_image(image_url = image_from_sentiment_url)


Keywords: Healing, Strength, Support
Prompt: Create an image that conveys the power of healing, strength, and support.
